In [1]:
import trainer
from utils import batchify
import json
import tensorflow as tf
import numpy as np

In [2]:
with open('SentimentAnalysis/full/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('SentimentAnalysis/full/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)

In [3]:
with open('107_SA/trainer_params.json', 'r') as inp: params = json.load(inp)
# In[12]:

my_trainer = trainer.Trainer(**params, fine_tune=True)

my_trainer.logger.info('Trainer params {}'.format(params))
# In[13]:


tf.reset_default_graph()
my_trainer.build(save_optimizer_var=False)

Trainer params {'model_configs': {'drop_e': 0.1, 'skip_connection': True, 'projection_dims': 512, 'char_cnn_options': {'n_highways': 2, 'layers': [[1, 16], [2, 16], [3, 32], [4, 64], [5, 128], [6, 256], [7, 512]]}, 'char_vocab_size': 729, 'vocab_size': 122289, 'rnn_layers': [{'drop_o': 0.2, 'drop_i': 0.2, 'units': 1024, 'wdrop': 0.4}, {'drop_o': 0.2, 'units': 1024, 'wdrop': 0.4}, {'drop_o': 0.2, 'units': 1024, 'wdrop': 0.4}], 'char_vec_size': 16}, 'bptt': 100, 'optimizer': {'params': {'weight_decay': 1.2e-06, 'beta2': 0.99, 'beta1': 0.8}, 'name': 'adamw'}, 'train_summary_dir': '107_SA/train_summary/', 'negative_samples': 10000, 'wdecay': 0.0, 'checkpoint_dir': '107_SA/checkpoints/', 'log_path': '107_SA/logs', 'beta': 1e-06, 'use_ema': True, 'test_summary_dir': '107_SA/test_summary/', 'clip_norm': 0.3, 'save_freq': 1000, 'alpha': 1e-06}


INFO:tensorflow:Restoring parameters from 107_SA/checkpoints/train/model.cpkt-73000


Restoring parameters from 107_SA/checkpoints/train/model.cpkt-73000


In [4]:
with open('SentimentAnalysis/full/train_word.npy','rb') as inp:
    train_word = np.load(inp)
with open('SentimentAnalysis/full/train_char.npy','rb') as inp:
    train_char = np.load(inp)
with open('SentimentAnalysis/full/test_word.npy','rb') as inp:
    valid_word = np.load(inp)
with open('SentimentAnalysis/full/test_char.npy','rb') as inp:
    valid_char = np.load(inp)

In [5]:
train_word = batchify(train_word, 22).T
train_char = batchify(train_char, 22).T
valid_word = batchify(valid_word, 11).T
valid_char = batchify(valid_char, 11).T

In [6]:
# Reset global step to zero
my_trainer.session.run(tf.assign(my_trainer.global_step, 0))

0

In [6]:
start_i = 0
lr = 5e-4
fine_tune_rate = [1/2.6, 1/3.6, 1/3.6, 1/2.6]
# fine_tune_rate = [1, 1, 1, 1]
print(fine_tune_rate)
for _ in range(20):
    my_trainer.train_dev_loop(train_word, train_char, valid_word, valid_char, lr, start_i=start_i, folder_test='test_sa', folder_train='train_sa', fine_tune_rate=fine_tune_rate)
    lr = lr * 0.9
    start_i = 0

[0.3846153846153846, 0.2777777777777778, 0.2777777777777778, 0.3846153846153846]


Step 73001: progress 100/9707,forward loss 5.13583, backward loss 5.14947, ppl 171.17, bpc 07.42, time 02.78
Step 73002: progress 201/9707,forward loss 5.05467, backward loss 5.03018, ppl 154.84, bpc 07.27, time 03.49
Step 73003: progress 250/9707,forward loss 4.99375, backward loss 5.30230, ppl 172.09, bpc 07.43, time 03.85
Step 73004: progress 347/9707,forward loss 4.97279, backward loss 4.98179, ppl 145.08, bpc 07.18, time 04.55
Step 73005: progress 447/9707,forward loss 4.85903, backward loss 4.88870, ppl 130.83, bpc 07.03, time 04.98
Step 73006: progress 543/9707,forward loss 4.75980, backward loss 5.03738, ppl 134.10, bpc 07.07, time 05.58
Step 73007: progress 644/9707,forward loss 5.02870, backward loss 5.09554, ppl 157.92, bpc 07.30, time 06.29
Step 73008: progress 747/9707,forward loss 5.01977, backward loss 5.07132, ppl 155.33, bpc 07.28, time 07.03
Step 73009: progress 853/9707,forward loss 4.86348, backward loss 5.11351, ppl 146.72, bpc 07.20, time 07.71
Step 73010: progres

Step 73076: progress 7233/9707,forward loss 4.23913, backward loss 4.32054, ppl 72.23, bpc 06.17, time 44.46
Step 73077: progress 7328/9707,forward loss 4.25864, backward loss 4.42818, ppl 76.97, bpc 06.27, time 44.87
Step 73078: progress 7425/9707,forward loss 4.29362, backward loss 4.28707, ppl 72.99, bpc 06.19, time 45.29
Step 73079: progress 7521/9707,forward loss 4.29479, backward loss 4.42068, ppl 78.08, bpc 06.29, time 45.71
Step 73080: progress 7622/9707,forward loss 4.14246, backward loss 4.25270, ppl 66.52, bpc 06.06, time 46.14
Step 73081: progress 7716/9707,forward loss 4.42967, backward loss 4.42951, ppl 83.90, bpc 06.39, time 46.56
Step 73082: progress 7772/9707,forward loss 4.44264, backward loss 4.37728, ppl 82.27, bpc 06.36, time 47.05
Step 73083: progress 7866/9707,forward loss 4.36242, backward loss 4.42343, ppl 80.88, bpc 06.34, time 47.47
Step 73084: progress 7970/9707,forward loss 4.24703, backward loss 4.25897, ppl 70.32, bpc 06.14, time 47.90
Step 73085: progres

Step 73132: progress 3003/9707,forward loss 4.23386, backward loss 4.32807, ppl 72.31, bpc 06.18, time 14.38
Step 73133: progress 3101/9707,forward loss 4.18874, backward loss 4.17706, ppl 65.56, bpc 06.03, time 14.80
Step 73134: progress 3141/9707,forward loss 4.35977, backward loss 4.08044, ppl 68.04, bpc 06.09, time 15.17
Step 73135: progress 3239/9707,forward loss 4.15695, backward loss 3.92967, ppl 57.01, bpc 05.83, time 15.60
Step 73136: progress 3341/9707,forward loss 4.25138, backward loss 4.43043, ppl 76.78, bpc 06.26, time 16.13
Step 73137: progress 3442/9707,forward loss 4.29593, backward loss 4.42827, ppl 78.42, bpc 06.29, time 16.57
Step 73138: progress 3542/9707,forward loss 4.32424, backward loss 4.30404, ppl 74.75, bpc 06.22, time 17.09
Step 73139: progress 3638/9707,forward loss 4.30895, backward loss 4.07552, ppl 66.17, bpc 06.05, time 17.51
Step 73140: progress 3728/9707,forward loss 4.23833, backward loss 4.13327, ppl 65.75, bpc 06.04, time 18.10
Step 73141: progres

Evaluate loss 4.863927841186523, time 5.007366418838501
Evaluate loss 4.882312774658203, time 5.253839492797852
Evaluate loss 5.111902236938477, time 5.499418258666992
Evaluate loss 4.94334077835083, time 5.746158599853516
Evaluate loss 4.668655872344971, time 5.993653297424316
Evaluate loss 5.062450408935547, time 6.239897966384888
Evaluate loss 4.899619102478027, time 6.487115144729614
Evaluate loss 4.609689712524414, time 6.733543872833252
Evaluate loss 4.7733235359191895, time 6.981052398681641
Evaluate loss 4.747367858886719, time 7.227316617965698
Evaluate loss 4.734652996063232, time 7.476441144943237
Evaluate loss 4.740447521209717, time 7.7221360206604
Evaluate loss 4.941105842590332, time 7.9689600467681885
Evaluate loss 4.751404285430908, time 8.215753316879272
Evaluate loss 4.957609176635742, time 8.461554288864136
Evaluate loss 5.104090690612793, time 8.707850933074951
Evaluate loss 4.952768802642822, time 8.955114603042603
Evaluate loss 4.825718879699707, time 9.201654195

...

Step 74711: progress 393/9707,forward loss 3.37082, backward loss 3.39417, ppl 29.44, bpc 04.88, time 01.72
Step 74712: progress 492/9707,forward loss 3.46764, backward loss 3.61047, ppl 34.43, bpc 05.11, time 02.14
Step 74713: progress 586/9707,forward loss 3.48939, backward loss 3.57730, ppl 34.24, bpc 05.10, time 02.55
Step 74714: progress 682/9707,forward loss 3.66360, backward loss 3.60481, ppl 37.87, bpc 05.24, time 02.97
Step 74715: progress 786/9707,forward loss 3.51063, backward loss 3.59577, ppl 34.92, bpc 05.13, time 03.42
Step 74716: progress 839/9707,forward loss 3.65059, backward loss 3.70612, ppl 39.58, bpc 05.31, time 03.69
Step 74717: progress 931/9707,forward loss 3.41475, backward loss 3.72362, ppl 35.49, bpc 05.15, time 04.10
Step 74718: progress 1025/9707,forward loss 3.54379, backward loss 3.63876, ppl 36.28, bpc 05.18, time 04.51
Step 74719: progress 1125/9707,forward loss 3.37707, backward loss 3.59051, ppl 32.58, bpc 05.03, time 04.95
Step 74720: progress 1228/

Step 74787: progress 7636/9707,forward loss 3.32273, backward loss 3.47685, ppl 29.96, bpc 04.90, time 33.37
Step 74788: progress 7737/9707,forward loss 3.47739, backward loss 3.60109, ppl 34.44, bpc 05.11, time 33.80
Step 74789: progress 7839/9707,forward loss 3.50957, backward loss 3.46153, ppl 32.64, bpc 05.03, time 34.24
Step 74790: progress 7932/9707,forward loss 3.52282, backward loss 3.44129, ppl 32.53, bpc 05.02, time 34.65
Step 74791: progress 8028/9707,forward loss 3.39856, backward loss 3.52602, ppl 31.89, bpc 05.00, time 35.07
Step 74792: progress 8130/9707,forward loss 3.55732, backward loss 3.55091, ppl 34.96, bpc 05.13, time 35.50
Step 74793: progress 8230/9707,forward loss 3.48941, backward loss 3.59727, ppl 34.58, bpc 05.11, time 35.94
Step 74794: progress 8325/9707,forward loss 3.62869, backward loss 3.41339, ppl 33.82, bpc 05.08, time 36.36
Step 74795: progress 8368/9707,forward loss 3.63666, backward loss 3.56148, ppl 36.56, bpc 05.19, time 36.60
Step 74796: progres

Step 74843: progress 3222/9707,forward loss 3.44496, backward loss 3.14591, ppl 26.99, bpc 04.75, time 14.15
Step 74844: progress 3325/9707,forward loss 3.61749, backward loss 3.63656, ppl 37.60, bpc 05.23, time 14.59
Step 74845: progress 3422/9707,forward loss 3.54502, backward loss 3.62781, ppl 36.10, bpc 05.17, time 15.02
Step 74846: progress 3520/9707,forward loss 3.44031, backward loss 3.44509, ppl 31.27, bpc 04.97, time 15.45
Step 74847: progress 3620/9707,forward loss 3.56648, backward loss 3.37967, ppl 32.24, bpc 05.01, time 15.88
Step 74848: progress 3717/9707,forward loss 3.62675, backward loss 3.43549, ppl 34.16, bpc 05.09, time 16.31
Step 74849: progress 3817/9707,forward loss 3.71623, backward loss 3.62563, ppl 39.29, bpc 05.30, time 16.74
Step 74850: progress 3919/9707,forward loss 3.85159, backward loss 3.46174, ppl 38.73, bpc 05.28, time 17.18
Step 74851: progress 4014/9707,forward loss 3.68612, backward loss 3.69893, ppl 40.15, bpc 05.33, time 17.59
Step 74852: progres

Evaluate loss 4.16758394241333, time 5.211565256118774
Evaluate loss 3.8913042545318604, time 5.459880828857422
Evaluate loss 4.131564140319824, time 5.706990957260132
Evaluate loss 4.131992340087891, time 5.954832315444946
Evaluate loss 3.853449821472168, time 6.201006174087524
Evaluate loss 3.915752410888672, time 6.449726343154907
Evaluate loss 3.870523452758789, time 6.696345806121826
Evaluate loss 4.029568672180176, time 6.945652723312378
Evaluate loss 3.939056396484375, time 7.192242622375488
Evaluate loss 4.093206882476807, time 7.438006162643433
Evaluate loss 3.9512434005737305, time 7.68675684928894
Evaluate loss 4.13140344619751, time 7.934244155883789
Evaluate loss 4.179928302764893, time 8.182583808898926
Evaluate loss 4.04465913772583, time 8.42891812324524
Evaluate loss 3.9590775966644287, time 8.674400329589844
Evaluate loss 4.010534286499023, time 8.921090126037598
Evaluate loss 4.107609748840332, time 9.169604063034058
Evaluate loss 3.958042860031128, time 9.4166164398

Step 74974: progress 6104/9707,forward loss 3.33336, backward loss 3.51672, ppl 30.72, bpc 04.94, time 26.58
Step 74975: progress 6146/9707,forward loss 3.30730, backward loss 3.38486, ppl 28.39, bpc 04.83, time 26.82
Step 74976: progress 6247/9707,forward loss 3.36704, backward loss 3.47503, ppl 30.60, bpc 04.94, time 27.26
Step 74977: progress 6347/9707,forward loss 3.77090, backward loss 3.48530, ppl 37.64, bpc 05.23, time 27.69
Step 74978: progress 6450/9707,forward loss 3.60466, backward loss 3.38583, ppl 32.96, bpc 05.04, time 28.14
Step 74979: progress 6553/9707,forward loss 3.37809, backward loss 3.34938, ppl 28.90, bpc 04.85, time 28.57
Step 74980: progress 6655/9707,forward loss 3.39837, backward loss 3.42219, ppl 30.27, bpc 04.92, time 29.01
Step 74981: progress 6755/9707,forward loss 3.42752, backward loss 3.56793, ppl 33.04, bpc 05.05, time 29.44
Step 74982: progress 6844/9707,forward loss 3.55100, backward loss 3.36640, ppl 31.78, bpc 04.99, time 29.84
Step 74983: progres